In [1]:
import numpy as np
import pandas as pd
from functools import reduce
# from imblearn.under_sampling import RandomUnderSampler
import random

In [2]:
adm = pd.read_csv('s3://athena-output-mimic/admissions/2022/04/01/0d87fbd0-5127-43aa-a6a9-74535f99b093.csv')
pats = pd.read_csv('s3://athena-output-mimic/patient_age/2022/04/01/af54b7c1-5c76-412e-adcc-54bf649d92fe.csv')
notes = pd.read_csv('s3://athena-output-mimic/noteevents/2022/04/01/a4ec2a40-3dd5-42fa-b6da-b22f3dc96880.csv')
scripts = pd.read_csv('s3://athena-output-mimic/prescriptions/2022/04/01/a1ef6eec-e3b0-47d0-be6d-42616c00547f.csv')

In [3]:
pats.head()

,subject_id,dob,dod
0,249,2075-03-13 00:00:00.000,NaN
1,250,2164-12-27 00:00:00.000,2188-11-22 00:00:00.000
2,251,2090-03-15 00:00:00.000,NaN
3,252,2078-03-06 00:00:00.000,NaN
4,253,2089-11-26 00:00:00.000,NaN


In [4]:
pats.nunique()

subject_id    46520
dob           32540
dod           12911
dtype: int64

In [5]:
adm.head()

,subject_id,hadm_id,admittime
0,22,165315,2196-04-09 12:26:00.000
1,23,152223,2153-09-03 07:15:00.000
2,23,124321,2157-10-18 19:34:00.000
3,24,161859,2139-06-06 16:14:00.000
4,25,129635,2160-11-02 02:06:00.000


In [6]:
age = pd.merge(pats.drop('dod', axis=1), adm, on='subject_id')

In [7]:
age.head()

,subject_id,dob,hadm_id,admittime
0,249,2075-03-13 00:00:00.000,116935,2149-12-17 20:41:00.000
1,249,2075-03-13 00:00:00.000,149546,2155-02-03 20:16:00.000
2,249,2075-03-13 00:00:00.000,158975,2156-04-27 15:33:00.000
3,250,2164-12-27 00:00:00.000,124271,2188-11-12 09:22:00.000
4,251,2090-03-15 00:00:00.000,117937,2110-07-27 06:46:00.000


In [8]:
date_convert = ['admittime', 'dob']

for date in date_convert:
    age[date] = pd.to_datetime(age[date])

In [9]:
def get_age(x):
    age = (x['admittime'].to_pydatetime() - x['dob'].to_pydatetime()).days / 365 
    return age

In [10]:
# Number that meet age filter

age['age'] = age.apply(get_age, axis=1)
age = age[age['age'].between(18, 85)].drop_duplicates().sort_values('subject_id')
age['subject_id'].nunique()

34458

In [11]:
filt_notes = notes[notes['subject_id'].isin(age['subject_id'])]
filt_notes = filt_notes.rename(columns={'chartdate': 'date'})
filt_notes = filt_notes[~filt_notes['hadm_id'].isna()]
filt_notes.head()

,subject_id,hadm_id,date,text
0,20181,198174.0,2173-09-01 00:00:00.000,Resp Care\nremains ett/vent support. able to w...
1,20133,109649.0,2159-05-21 00:00:00.000,Condition Update A:\nPlease refer to careview ...
2,20116,101379.0,2127-03-29 00:00:00.000,CCU NPN: please see flowsheet for objective da...
3,20119,152965.0,2145-09-10 00:00:00.000,"Neuro: AAOx3, MAEx4, OOB to chair and was unst..."
4,20236,154430.0,2130-09-04 00:00:00.000,Respiratory Care\nPt was extubated today at 10...


In [12]:
filt_scripts = scripts[scripts['subject_id'].isin(age['subject_id'])]
filt_scripts = filt_scripts.rename(columns={'startdate': 'date'}).drop('enddate', axis=1)
filt_scripts = filt_scripts[~filt_scripts['hadm_id'].isna()]
filt_scripts.head()

,subject_id,hadm_id,date,drug
0,4,185777,2191-03-16 00:00:00.000,Benzonatate
1,4,185777,2191-03-16 00:00:00.000,Dextromethorphan-Guaifenesin (Sugar Free)
2,68,170467,2173-12-25 00:00:00.000,Guaifenesin-Dextromethorphan
3,85,112077,2167-07-26 00:00:00.000,Benzonatate
4,85,112077,2167-07-26 00:00:00.000,Guaifenesin-Dextromethorphan


In [13]:
conc = pd.concat([filt_notes, filt_scripts]).sort_values('subject_id')
conc.shape

(162777, 5)

In [14]:
# All subject id's with at least a cough prescription, or a mention of cough in clinical notes
all_df = conc.copy()
all_cough = all_df['subject_id'].unique().tolist()

In [15]:
len(all_cough)

23901

In [16]:
# Get subjects with at least three hospital admissions

sub_ids = conc.groupby('subject_id')['hadm_id'].nunique()[conc.groupby('subject_id')['hadm_id'].nunique() >= 3].index.to_list()
len(sub_ids)

1201

In [17]:
conc = conc[conc['subject_id'].isin(sub_ids)]
conc.shape

(30796, 5)

In [18]:
conc['date'] = pd.to_datetime(conc['date'])
conc = conc.sort_values(['subject_id', 'hadm_id', 'date'])
# conc = conc.sort_values(['subject_id', 'date'])
conc.head(10)

,subject_id,hadm_id,date,text,drug
183773,36,122659.0,2131-05-17,Pt returned to [**Location 204**] s/p chest ex...,NaN
23251,36,122659.0,2131-05-21,ROS:\n\nNeuro: Sleepy. Arouses w/ease. Denies ...,NaN
23412,36,122659.0,2131-05-21,Nursing Progress Note:\nNeuro: Pt lethargic. ...,NaN
23429,36,122659.0,2131-05-22,Respiratory Care\nPt remains comfortable with ...,NaN
23413,36,122659.0,2131-05-22,"npn 23:00-07:00\n\ns/p CABGx5 [**2131-5-3**], ...",NaN
23269,36,122659.0,2131-05-23,"Neuro: alert and oriented X3, has some visual ...",NaN
44402,36,122659.0,2131-05-25,Admission Date: [**2131-5-12**] ...,NaN
44403,36,165660.0,2134-05-20,Admission Date: [**2134-5-10**] ...,NaN
23384,36,182104.0,2131-05-04,a paced->nsr 60's-80's. pacer off as occasiona...,NaN
131170,109,102024.0,2142-06-03,: Ms. [**Known lastname 406**] is a 24 y.o. fe...,NaN


In [19]:
date_diff = conc[['subject_id', 'hadm_id', 'date']].drop_duplicates(subset='hadm_id')
date_diff = date_diff.sort_values(['subject_id', 'date'])
date_diff['shift_date'] = date_diff['date'].shift(2)
date_diff.head()

,subject_id,hadm_id,date,shift_date
23384,36,182104.0,2131-05-04,NaT
183773,36,122659.0,2131-05-17,NaT
44403,36,165660.0,2134-05-20,2131-05-04
176633,109,183350.0,2137-11-05,2131-05-17
176690,109,128755.0,2138-04-17,2134-05-20


In [20]:
date_diff['diff'] = date_diff['date'] - date_diff['shift_date']

cc = date_diff[date_diff['diff'] <= pd.Timedelta(120, 'd')]
cc_sub = cc['subject_id'].unique().tolist()
len(cc_sub)

780

In [21]:
# cc_ext = date_diff[date_diff['diff'] <= pd.Timedelta(365*2, 'd')]
# cc_extSub = cc_ext['subject_id'].unique().tolist()
# len(cc_extSub )

In [22]:
# cc_sub

In [23]:
# all_cough

In [24]:
non_cc = [id for id in all_cough if id not in cc_sub]

In [25]:
len(non_cc)

23121

In [26]:
random.seed(2021)
non_cc = random.sample(non_cc, len(cc_sub))

In [27]:
len(non_cc)

780

In [110]:
cc_df = pd.DataFrame({'subject_id': cc_sub, 'target': 1.0})

In [111]:
noncc_df = pd.DataFrame({'subject_id': non_cc, 'target': 0})

In [118]:
cohort = pd.concat([cc_df, noncc_df])
cohort.to_csv('cohort.csv')

In [119]:
cohort.head()

,subject_id,target
0,109,1.0
1,188,1.0
2,222,1.0
3,236,1.0
4,305,1.0


In [120]:
cohort['subject_id'].nunique()

1560